In [60]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [61]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random

In [62]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
all_scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    all_scenarios[i] = df_helper
end

In [63]:
W = 250
hours = 24
Random.seed!(123)
selected_scenarios = rand(1:n_scenarios, W)

scenarios = Dict()
counter = 1
for i in selected_scenarios
    scenarios[counter] = all_scenarios[i]
    counter += 1
end

beta = 0.99
alpha = 0.95

0.95

In [64]:
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for hour
@variable(model, p_DA[1:hours])
@variable(model, delta[1:W,1:hours])
@variable(model, zeta)
@variable(model, eta[1:W] >= 0)

# Define the objective function
@objective(model, Max, sum(1/W .*
(scenarios[i][hour,"price"] * p_DA[hour]
+ scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours)
+ beta  * (zeta - (1/(1-alpha)) * sum(1/W .* eta[i] for i in 1:W)))

# Define the constraints
@constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
@constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
@constraint(model, [i in 1:W], -sum((scenarios[i][hour,"price"] * p_DA[hour]
+ scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for hour in 1:hours) + zeta - eta[i] <= 0)


# Solve the optimization problem
optimize!(model)

exp_profit = sum(1/W .*
(scenarios[i][hour,"price"] * value.(p_DA[hour])
+ scenarios[i][hour,"price"] * value.(delta[i, hour])*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours)

CVaR = beta  * (zeta - (1/(1-alpha)) * sum(1/W .* eta[i] for i in 1:W))

# Print the termination status
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    
    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))
    println("Expected profit: ", exp_profit)
    println("CVaR: ", CVaR)
else
    println("No optimal solution found")
end

In [65]:
println("p_DA: ", value.(p_DA))
println("delta: ", value.(delta))

In [66]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], p_DA = Float64[])

# make a table with the results
for hour in 1:hours
    push!(result_df, [hour, value(p_DA[hour])])
end

profit_df = DataFrame(scenario = Int[], profit = Float64[])

for i in 1:W
    profit = 0
    for hour in 1:hours
        profit += (scenarios[i][hour,"price"] * value(p_DA[hour])
        + scenarios[i][hour,"price"] * value(delta[i, hour])*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"])))
    end
    push!(profit_df, [i, profit])
end

In [67]:
pretty_table(result_df, backend = Val(:latex))